In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 09d56f18-323d-4759-97a2-67d74224c8ad, 3, Finished, Available, Finished)

In [2]:
%%sql
create table if not exists LakeHouse_2.gold_product
(
Product_ID long,
Product_Category string,
Product string,
Created_TS timestamp,
Modified_TS timestamp
)
using delta

StatementMeta(, 09d56f18-323d-4759-97a2-67d74224c8ad, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
Max_Date=spark.sql("""
select coalesce(max(Modified_TS),'1900-01-01') from LakeHouse_2.gold_product
""").first()(0)

StatementMeta(, 09d56f18-323d-4759-97a2-67d74224c8ad, 6, Finished, Available, Finished)

In [7]:
df_bronze = spark.sql(f"""
    SELECT DISTINCT
        Product_Category,
        Product
    FROM LakeHouse_2.bronze_sales
    WHERE Modified_TS > '{Max_Date}'
""")


StatementMeta(, 09d56f18-323d-4759-97a2-67d74224c8ad, 9, Finished, Available, Finished)

In [8]:
Max_ID=spark.sql("select coalesce(max(Product_ID),0) from LakeHouse_2.gold_product").first()[0]

StatementMeta(, 09d56f18-323d-4759-97a2-67d74224c8ad, 10, Finished, Available, Finished)

In [10]:
df_final=df_bronze.withColumn("Product_ID",monotonically_increasing_id()+Max_ID+1)

StatementMeta(, 09d56f18-323d-4759-97a2-67d74224c8ad, 12, Finished, Available, Finished)

In [11]:
df_final.createOrReplaceTempView("ViewProduct")

StatementMeta(, 09d56f18-323d-4759-97a2-67d74224c8ad, 13, Finished, Available, Finished)

In [12]:
%%sql
Merge into LakeHouse_2.gold_product as gp
using ViewProduct as vp
on gp.Product=vp.Product and gp.Product_Category=vp.Product_Category
when matched then 
update SET
gp.Modified_TS	=	current_timestamp()	

when not matched then insert
(
gp.Product_ID	,
gp.Product_Category	,
gp.Product	,
gp.Created_TS	,
gp.Modified_TS	
)
values
(
vp.Product_ID	,
vp.Product_Category	,
vp.Product	,
current_timestamp(),
current_timestamp
)

StatementMeta(, 09d56f18-323d-4759-97a2-67d74224c8ad, 14, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>